In [4]:
#PREPROCESS_somascan_raw_data_STEP1
#
#Preprocess for the proteomics data. Mainly, only considering Human proteins.
#Thus, we will remove: 
#Non-human protein, Spuriomer, Spurimer, HCE, Non-Biotin, Non-Cleavable, etc.
#
#In order to address the duplicates (However, different isoform),
#Feature = [PROTEIN SYMBOL]_[BARCODE]
#
#Intended inputs: somascan_data_parse_ready.tsv

In [12]:
import pandas as pd

In [29]:
def parse_from_anml_data(data_file):
    data_open = open(data_file,'r')
    data_readlines = data_open.readlines()
    
    sample_ID_list = []
    protein_list =[]
    data_dict = {}
       
    temp_seq_ID_list = data_readlines[0].split('\t')
    temp_protein_list = data_readlines[7].split('\t')
    temp_species_list = data_readlines[8].split('\t')
    target_type_list = data_readlines[10].split('\t')
    
    
    #make protein_list without mouse proteins
    for i in range(len(temp_species_list)):
        protein_name = temp_protein_list[i]
        species_type = temp_species_list[i]
        target_type = target_type_list[i]

        if species_type == "Human":            
            if target_type == "Protein":
                if protein_name != "None":  
                    seq_ID = temp_seq_ID_list[i]
                    protein_name_with_seqID = '%s_%s' % (protein_name, seq_ID)
                    protein_list.append(protein_name_with_seqID)

    #make dict[sampleid, protein] = value
    for i in range(21, len(data_readlines)):
        read = data_readlines[i]
        read = read.replace('\n','')
        token = read.split('\t')
        
        sample_ID = token[0]
        sample_ID_list.append(sample_ID)
        sample_specific_protein_dict_list = []

        for j in range(1, len(token)):
            
            protein_symbol = temp_protein_list[j]
            species_type = temp_species_list[j]
            target_type = target_type_list[j]
            seq_ID = temp_seq_ID_list[j]
            
            value = token[j]
            
            if species_type == "Human": #do only human proteins
                if protein_symbol != "None":
                    if target_type == "Protein":
                        if protein_symbol not in sample_specific_protein_dict_list:    
                            protein_name_with_seqID = '%s_%s' % (protein_symbol, seq_ID)
                            data_dict[sample_ID, protein_name_with_seqID] = value
                            sample_specific_protein_dict_list.append(protein_name_with_seqID)
                    
    return sample_ID_list, protein_list, data_dict

In [30]:
def make_output_from_dict(sample_ID_list, protein_list, data_dict, output_dir):
    output_txt = open(output_dir,'w')
    
    #write header
    output_txt.write('sample_ID')
    for protein_symbol in protein_list:
        output_txt.write('\t%s' % protein_symbol)
    output_txt.write('\n')
    
    #write main
    for sample_ID in sample_ID_list:
        output_txt.write(sample_ID)
        for protein_symbol in protein_list:
            value = data_dict[sample_ID, protein_symbol]
            output_txt.write('\t%s' % value)
        output_txt.write('\n')
    
    
    output_txt.close()

In [31]:
data_file = "../../../raw_data/RA_somascan/MAYO-216978_v4.1_EDTAPlasma_20210905/somascan_data_parse_ready.tsv"
output_file = '../../../preprocessed_data/proteomics/somascan_anml.tsv'

sample_ID_list, protein_list, data_dict = parse_from_anml_data(data_file)


In [33]:
make_output_from_dict(sample_ID_list, protein_list, data_dict, output_file)

In [34]:
#transpose_data
data_df = pd.read_csv(output_file, sep="\t", index_col=0)
data_df = data_df.T
data_df.to_csv('../../../preprocessed_data/proteomics/somascan_anml.T.tsv', sep="\t")